In [3]:
import os
import numpy as np
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from xgboost import XGBClassifier

data_dir = "./abide"

X = []
y = []

for subject in os.listdir(data_dir):
    subject_path = os.path.join(data_dir, subject)
    
    if os.path.isdir(subject_path):
        for file in os.listdir(subject_path):
            if file.endswith("AAL116_correlation_matrix.mat"):
                mat_data = scipy.io.loadmat(os.path.join(subject_path, file))
                
                correlation_matrix = mat_data.get("data")
                if correlation_matrix is None:
                    continue
                
                X.append(correlation_matrix.flatten())

                label = 0 if "control" in subject.lower() else 1
                y.append(label)

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = XGBClassifier(n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    random_state=42,
    use_label_encoder=False,   # new versions warn about label encoding
    eval_metric='logloss')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Test Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/home/hice1/crunch40/.jupyterhub/lib64/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:04:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Test Accuracy: 0.6390

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.66      0.67       116
           1       0.58      0.62      0.60        89

    accuracy                           0.64       205
   macro avg       0.63      0.64      0.64       205
weighted avg       0.64      0.64      0.64       205

